In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import os
import time

In [7]:
%%time 

# Abriendo los archivos .csv
folderpath =  r"data"
print("Files and directories in a specified path:")
files = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
data\endes_2021_programas_sociales_hogar.csv
data\endes_2021_RE223132.csv
data\endes_2021_RE516171.csv
data\endes_2021_RE758081.csv
data\endes_2021_REC84DV.csv
data\endes_2021_REC91.csv
data\endes_2021_REC93DVdisciplina.csv
data\endes_2021_REC95.csv
data\endes_2021_RECH23.csv
Wall time: 4.99 ms


### Import csv using dask


In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

Client Scheduler: tcp://127.0.0.1:54276 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 12.68 GB


In [5]:
import dask.dataframe as dd

In [8]:
#guardamos los dask dataframes en una lista
dfs = []

for file in files:
    
    df = dd.read_csv(file, dtype=str)
    dfs.append(df)

In [9]:
#asignamos nombres a los dataframes
programas_sociales = dfs[0]
nacimiento = dfs[1]
nupcialidad_fecundidad = dfs[2]
sida_condon = dfs[3]
mortalidad_violencia = dfs[4]
MEF = dfs[5] 
disciplina_infantil = dfs[6]
inmunizacion_salud = dfs[7]
vivienda = dfs[8]

In [10]:
# identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud


#### Merging dfs

In [11]:
base_total = nacimiento.merge(nupcialidad_fecundidad, on = ['CASEID', 'ID1'], how ='left').merge(sida_condon, on = ['CASEID', 'ID1'], how ='left').merge(mortalidad_violencia, on = ['CASEID', 'ID1'], how ='left').merge(MEF, on = ['CASEID', 'ID1'], how ='left').merge(disciplina_infantil, on = ['CASEID', 'ID1'], how ='left').merge(inmunizacion_salud, on = ['CASEID', 'ID1'], how ='left')

In [13]:
df_final = base_total.compute()

In [14]:
df_final.dtypes

ID1         object
CASEID      object
V201        object
V202        object
V203        object
             ...  
S466DC12    object
S466MC12    object
S466YC12    object
S466WC12    object
S466HC12    object
Length: 1272, dtype: object

In [15]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = df_final[variables]
data_final.shape

Wall time: 673 ms


(43137, 27)

In [16]:
for i in variables:
        if data_final[i].dtype == 'O':
            data_final[i] = data_final[i].str.replace(" ", "")
            # Reemplazando los missing values por NaN
            data_final[i] = data_final[i].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            data_final[i] =  np.floor(pd.to_numeric(data_final[i], errors='coerce').astype('Int64'))

In [17]:
data_final.dtypes

ID1        Float64
SREGION    Float64
SPROVIN    Float64
SDISTRI    Float64
CASEID     Float64
S108N      Float64
S119       Float64
S119D      Float64
V218       Float64
V213       Float64
V225       Float64
S45D4      Float64
S45D5      Float64
S45P4      Float64
S45P5      Float64
S45B0      Float64
S45B1      Float64
S45B2      Float64
S45B3      Float64
S45F1      Float64
S45F2      Float64
V501       Float64
V714       Float64
D104       Float64
D106       Float64
D107       Float64
D108       Float64
dtype: object

#### Creación de por lo menos dos variables

*Las variables de interés son las siguientes:*

D104: Hubo violencia emocional | 0=No, 1=Sí

D106: Experimentó cualquier violencia física menos severa | 0=No, 1=Sí

D107: Experimentó cualquier violencia física severa | 0=No, 1=Sí

D108: Experimentó cualquier violencia sexual | 0=No, 1=Sí



In [18]:
# Agregando los datos para evaluar cuántas mujeres han sido víctimas de violencia en al menos una de sus modalidades
data_final['violencia'] = data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108']

# Contabilizando lo encontrado
data_final['violencia'].value_counts()

0.0    18395
1.0     4372
2.0     2621
3.0     1598
4.0      817
Name: violencia, dtype: Int64

In [19]:
# Creando variable dummy 

data_final['y'] = data_final['violencia']
data_final.loc[(data_final['y'] > 0), 'y'] = 1
data_final['y'].value_counts()

0.0    18395
1.0     9408
Name: y, dtype: Int64